In [1]:
import pyspark
from pyspark.sql.types import IntegerType

spark = pyspark.sql.SparkSession.builder \
    .master("local") \
    .appName("movies") \
    .getOrCreate()

In [3]:
df = spark.read.csv(path="/home/jovyan/movielens/ratings.csv", header=True)
df = df.withColumn("rating", df["rating"].cast(IntegerType()))

In [4]:
averages = df \
    .groupBy("user_id") \
    .avg("rating") \
    .select("*")

In [5]:
subtracted = averages \
    .join(df, df["user_id"] == averages["user_id"]) \
    .select(df["user_id"], "avg(rating)", "rating", "movie_id") \
    .collect()

In [16]:
users = {}

for user in subtracted:
    user_id = user["user_id"]
    movie_id = user["movie_id"]
    rating = user["rating"]
    average = user["avg(rating)"]
    if user_id not in users:
        users[user_id] = {
            "average": average,
            movie_id: {
                "rating": rating,
                "diff": rating - average
            }
        }
    else:
        users[user_id][movie_id] = {
            "rating": rating,
            "diff": rating - average
        }

6040
